In [237]:
import pandas as pd
import numpy as np
import DataLoad as data
from sklearn import linear_model

In [238]:
pd.options.mode.chained_assignment = None
sessions_df = data.load_sessions_data()
users_df = data.load_users_data()
products_df = data.load_products_data()
cleared_sessions_df = sessions_df.loc[ (~sessions_df['user_id'].isna()) & (~sessions_df['product_id'].isna())]

In [239]:
def get_buys(uid):
    drop_list = []
    tmp = cleared_sessions_df.loc[(cleared_sessions_df['user_id'] == uid) & (cleared_sessions_df['event_type_BUY_PRODUCT'] == 1)]
    i = -1
    for index, row in tmp.iterrows():
        if index in drop_list:
            continue
        i = i + 1
        count = 1
        sum = row['offered_discount']
        j = i
        tmp2 = tmp[(i + 1):]
        for index2, row2 in tmp2.iterrows():
            j = j + 1
            if row['product_id'] == row2['product_id']:
                count = count + 1
                sum = sum + row2['offered_discount']
                tmp.drop(tmp.index[j], inplace=True)
                j = j - 1
                drop_list.append(index2)
        if count > 1:
             tmp.at[index, 'offered_discount'] = sum / count

    return tmp

def get_product_category(pid):
    tmp = products_df.loc[((products_df['product_id']).isin(pid))]
    tmp = tmp.drop(['product_name', 'price', 'product_id'], axis=1)
    return tmp


In [246]:

users = users_df['user_id']
reg_list = []
for id in users:
    reg = linear_model.LinearRegression()
    user_buys = get_buys(id)
    if len(user_buys.index) > 0:
        reg.fit(get_product_category(user_buys['product_id']), user_buys['offered_discount'])
        reg_list.append([id, reg])
    else:
        reg_list.append([id, 0])

[array([20.]),
 array([17.9877204]),
 array([15.]),
 0,
 array([14.23333333]),
 array([20.]),
 array([11.09550562]),
 array([20.]),
 array([14.54296875]),
 0,
 array([10.]),
 array([10.]),
 array([20.]),
 array([20.]),
 array([13.75]),
 array([20.]),
 array([20.]),
 array([15.]),
 array([20.]),
 0,
 array([15.]),
 array([14.]),
 array([20.]),
 array([20.]),
 0,
 array([15.]),
 array([11.91666667]),
 array([20.]),
 array([15.]),
 array([20.]),
 0,
 array([10.]),
 array([15.]),
 array([18.05825243]),
 array([15.]),
 array([20.]),
 array([20.]),
 array([16.8]),
 array([20.]),
 0,
 array([10.]),
 array([20.]),
 array([20.]),
 array([20.]),
 array([10.]),
 array([20.]),
 array([10.]),
 array([20.]),
 array([20.]),
 array([20.]),
 array([15.]),
 array([20.]),
 array([16.66666667]),
 array([10.]),
 array([20.]),
 array([20.]),
 array([20.]),
 array([20.]),
 array([11.12962963]),
 array([16.125]),
 array([12.1953125]),
 array([20.]),
 array([14.28571429]),
 0,
 array([20.]),
 0,
 array([10.]),

In [ ]:
a = []
for i in users.index:
    if reg_list[i][1] == 0:
        a.append(0)
    else:
        a.append(reg_list[i][1].predict(get_product_category([1311])))
a
